In [1]:
import spiketoolkit as st
import spikesorters as ss
import spikeextractors as se
import numpy as np
import spikemetrics.metrics as metrics
from spikemetrics.utils import Epoch

18:21:59 [I] klustakwik KlustaKwik2 version 0.2.6


In [2]:
# workshop_path = "/Users/colehurwitz/spiketutorials/Spike_sorting_workshop_2019/"
workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")


Could not load plane information. Assuming probe is in yz plane


In [3]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [4]:
sorting = ss.run_klusta(recording)

Klusta units are not unique! Using unique unit ids


In [5]:
recording.frame_to_time(recording.get_num_frames())

30.0

In [6]:
metric_info = st.validation.validation_tools.get_quality_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_features_props=False, verbose=False, seed=0)

spike_times = metric_info[0]
spike_clusters = metric_info[1]
amplitudes = metric_info[2]
pc_features = metric_info[3]
pc_feature_ind = metric_info[4]

In [7]:
metric_calculator = st.validation.qualitymetrics.MetricCalculator()

In [8]:
params = metric_calculator.get_default_params_dict()

In [9]:
start_epoch = Epoch("start", 0, 15)
end_epoch = Epoch("end", 15, 30)
epochs = [start_epoch, end_epoch]

In [10]:
metrics_pd = metrics.calculate_metrics(spike_times=spike_times, spike_clusters=spike_clusters, amplitudes=amplitudes, 
                                       pc_features=pc_features, pc_feature_ind=pc_feature_ind, params=params, epochs = epochs, cluster_ids=sorting.get_unit_ids())

Calculating isi violations
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating presence ratio
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating firing rate
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating amplitude cutoff
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating PC-based metrics
 ▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 19% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░ 33% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░ 41% 

...

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░ 89% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating silhouette score
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating drift metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered


In [11]:
metric_calculator.store_all_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, \
                                        max_num_pca_waveforms=np.inf, save_features_props=False)


In [12]:
epoch_tuples = [(0, 15), (15, 30)]
epoch_names = ["start", "end"]

In [13]:
firing_rates = metric_calculator.compute_firing_rates(unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names)

num_spikes = metric_calculator.compute_num_spikes(unit_ids=None, epoch_tuples=epoch_tuples)

isi_violations = metric_calculator.compute_isi_violations(isi_threshold=0.0015, min_isi=0.000166, unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names)

presence_ratios = metric_calculator.compute_presence_ratios(unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names)

max_drifts, cumulative_drifts = metric_calculator.compute_drift_metrics(drift_metrics_interval_s=51, drift_metrics_min_spikes_per_interval=10, unit_ids=None, \
                                                                        epoch_tuples=epoch_tuples)
silhouette_scores = metric_calculator.compute_silhouette_score(unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

isolation_distances = metric_calculator.compute_isolation_distances(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

l_ratios = metric_calculator.compute_l_ratios(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

d_primes = metric_calculator.compute_d_primes(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

nn_hit_rates, nn_miss_rates = metric_calculator.compute_nn_metrics(num_channels_to_compare=13, max_spikes_for_unit=500, max_spikes_for_nn=10000, n_neighbors=4, unit_ids=None, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 11% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░ 67% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


In [14]:
def nan_equal(a,b):
    try:
        np.testing.assert_equal(a,b)
    except AssertionError:
        return False
    return True

In [15]:
metrics_dict = metric_calculator.get_metrics_dict()

print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['firing_rate']]).flatten(), np.asarray(metrics_pd['firing_rate'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['num_spikes']]).flatten(), np.asarray(metrics_pd['num_spikes'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['isi_viol']]).flatten(), np.asarray(metrics_pd['isi_viol'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['presence_ratio']]).flatten(), np.asarray(metrics_pd['presence_ratio'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['max_drift']]).flatten(), np.asarray(metrics_pd['max_drift'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['cumulative_drift']]).flatten(), np.asarray(metrics_pd['cumulative_drift'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['silhouette_score']]).flatten(), np.asarray(metrics_pd['silhouette_score'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['isolation_distance']]).flatten(), np.asarray(metrics_pd['isolation_distance'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['l_ratio']]).flatten(), np.asarray(metrics_pd['l_ratio'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['d_prime']]).flatten(), np.asarray(metrics_pd['d_prime'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['nn_hit_rate']]).flatten(), np.asarray(metrics_pd['nn_hit_rate'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[1]) for epoch_metrics in metrics_dict['nn_miss_rate']]).flatten(), np.asarray(metrics_pd['nn_miss_rate'])))

True
True
True
True
True
True
True
True
True
True
True
True


In [16]:
metric_calculator.metrics

{'firing_rate': [(('start', 0, 15),
   array([ 2.26827336,  5.20368594, 15.34420214,  1.53442021,  2.20155944,
           1.46770629, 11.20793896, 13.94320977,  3.20226827,  4.26969103,
           0.66713922,  9.00637952, 16.21148313,  2.00141767, 19.88074886,
           2.53512905,  4.4698328 , 11.20793896,  3.33569612,  6.60467831,
           7.20510362,  5.40382771,  1.40099237, 11.67493641, 15.2107743 ,
          13.47621232, 11.74165034])),
  (('end', 15, 30),
   array([ 2.60109463,  3.80159984, 13.40564154,  1.66736835,  4.20176824,
           0.86703154, 10.33768378, 15.40648356,  5.33557872,  4.06837878,
           0.86703154,  8.47023122, 15.6732625 ,  2.26762096, 16.00673617,
           3.06795777,  4.73532612,  8.27014702,  2.20092622,  6.4693892 ,
           9.00378909,  5.66905239,  1.26719995, 11.73827319, 13.73911521,
          12.53861   , 11.27141005]))],
 'num_spikes': [(('0', 0, 15),
   array([ 34.,  78., 230.,  23.,  33.,  22., 168., 209.,  48.,  64.,  10.,
        